In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

26422272it [00:14, 1780988.94it/s]                              


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



29696it [00:00, 49768.79it/s]                           


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



4422656it [00:02, 2117429.77it/s]                             


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



6144it [00:00, 6289920.37it/s]          


Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [3]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataLoader = DataLoader(test_data,batch_size=batch_size)

In [7]:
for X,y in test_dataLoader:
    print("Shape of X [N,C,H,W] = ",X.shape)
    print("Shape of y [N] = ",y.shape)
    break

Shape of X [N,C,H,W] =  torch.Size([64, 1, 28, 28])
Shape of y [N] =  torch.Size([64])


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


### Practicing super

In [9]:
class Rectangle:
    def __init__(self,length,width):
        self.length = length
        self.width = width
    def area(self):
        return self.length* self.width
    def perimeter(self):
        return 2*(self.length+self.width)


In [11]:
class Square(Rectangle):
    def __init__(self,side):
        super().__init__(side,side)


In [ ]:
class Cube(Square):
    def surface_area(self):
        face_area = super().area()
        return face_area*6
    

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)
        

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

In [15]:
def train(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device),y.to(device)
        pred = model(X)
        loss = loss_fn(pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss,current = loss.item(),batch*len(X)
            print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [16]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataLoader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Loss: 2.292267 [    0/60000]
Loss: 2.282772 [ 6400/60000]
Loss: 2.261576 [12800/60000]
Loss: 2.265767 [19200/60000]
Loss: 2.237016 [25600/60000]
Loss: 2.221808 [32000/60000]
Loss: 2.231615 [38400/60000]
Loss: 2.196611 [44800/60000]
Loss: 2.189864 [51200/60000]
Loss: 2.168476 [57600/60000]
Test Error: 
 Accuracy: 44.2%, Avg loss: 2.155180 

Epoch 2
-------------------------------
Loss: 2.158058 [    0/60000]
Loss: 2.156250 [ 6400/60000]
Loss: 2.092738 [12800/60000]
Loss: 2.119971 [19200/60000]
Loss: 2.059889 [25600/60000]
Loss: 2.006485 [32000/60000]
Loss: 2.037144 [38400/60000]
Loss: 1.952185 [44800/60000]
Loss: 1.960105 [51200/60000]
Loss: 1.900152 [57600/60000]
Test Error: 
 Accuracy: 54.6%, Avg loss: 1.884957 

Epoch 3
-------------------------------
Loss: 1.905885 [    0/60000]
Loss: 1.892399 [ 6400/60000]
Loss: 1.765257 [12800/60000]
Loss: 1.820309 [19200/60000]
Loss: 1.707045 [25600/60000]
Loss: 1.651749 [32000/60000]
Loss: 1.684406 [38400/